# epymorph dev log 2023-07-11

Demonstration of the Initializer prototype quickly hacked into place for Alex's SDH IPM.

(Maybe this is a good candidate for the general initialization system?)

## Initializers

Looking ahead: know that we will have to construct the Simulation ourselves; the `run` command isn't capable of using Initializers yet.

First, construct the SDH builder. It starts with the `DefaultInitializer` which initializes compartments like the run command does: taking params to seed an infection of a certain size in a single location (by index).

In [5]:
from epymorph.data.ipm.sdh import sirhBuilder as SdhBuilder

# Constructing the SDH builder.
sdh_builder = SdhBuilder()

sdh_builder.initializer.__class__.__name__

'DefaultInitializer'

But we can override the initializer by providing an alternative `Initializer` instance. There is a small suite of implementations already, but you can always extend `Initializer` yourself to implement custom logic.

NOTE: Initializer constructor arguments can be given as param values instead. (But the constructor value, if given, has priority.)

In [6]:
from epymorph.data.ipm.initializer import BottomLocsInitializer, IndexedLocsInitializer, LabeledLocsInitializer, RandomLocsInitializer, TopLocsInitializer

# Infect 10,000 people in 10 randomly chosen locations.
RandomLocsInitializer(
    infection_seed_size=10_000,
    infection_locations=10
)

# Infect 10,000 people in these 4 location by index.
IndexedLocsInitializer(
    infection_seed_size=10_000,
    infection_locations=[46, 104, 154, 2445]
)

# Infect 10,000 people in these 4 locations by label.
LabeledLocsInitializer(
    infection_seed_size=10_000,
    infection_locations=['040130506101', '040130610161', '040130405211', '040138160001']
)

# Infect 10,000 people in the 10 locations with the least median age
BottomLocsInitializer(
    infection_seed_size=10_000,
    infection_locations=10,
    infection_attribute='median_age'
)

# Infect 10,000 people in the 10 locations with the highest population
TopLocsInitializer(
    infection_seed_size=10_000,
    infection_locations=10,
    infection_attribute='population'
)


## Running a sim

Now that we've seen examples of the available initializers, let's put together a working simulation:

In [7]:
from datetime import date
from epymorph.simulation import Simulation
from epymorph.data import geo_library, mm_library
from epymorph.data.ipm.initializer import TopLocsInitializer

sdh_builder = SdhBuilder()
sdh_builder.initializer = TopLocsInitializer(
    infection_seed_size=10_000,
    infection_locations=10,
    infection_attribute='population'
)

sim = Simulation(
    geo=geo_library['maricopa_cbg_2019'](),
    ipm_builder=sdh_builder,
    mvm_builder=mm_library['centroids']()
)

out = sim.run(
    param={
        'alpha': [0.1, 0.3, 0.2],
        'theta': 0.1,
        'gamma': 0.1000022,
        'phi': 40,
        'move_control': 0.9,
        'infection_duration': 4.0,
        'immunity_duration': 90.0,
        'hospitalization_duration': 14.0,
    },
    start_date=date(2019, 1, 1),
    duration_days=1
)

# This is not very interesting output, but you get the idea.
out.prevalence[0, 0:10]

Compiled predef in 225.598 ms
Selected 10 locations by geo[population]:
   labels: ['040132168472' '040136103001' '040136109002' '040134226461'
 '040130822031' '040130506101' '040138160001' '040138152001'
 '040130405211' '040130610161']
  indices: [1389 2017 2037 1969  383  104 2445 2427   46  154]
   values: [6018 6235 6606 6867 7419 7808 9708 7788 8898 7929]


array([[1791,    0,    0,    0],
       [2007,    0,    0,    0],
       [1264,    0,    0,    0],
       [1139,    0,    0,    0],
       [1487,    0,    0,    0],
       [2501,    0,    0,    0],
       [2051,    0,    0,    0],
       [1356,    0,    0,    0],
       [1838,    0,    0,    0],
       [ 647,    0,    0,    0]])

And here's a version passing args via params, just to see what that looks like.

In [8]:
from datetime import date
from epymorph.simulation import Simulation
from epymorph.data import geo_library, mm_library
from epymorph.data.ipm.initializer import TopLocsInitializer

sdh_builder = SdhBuilder()
sdh_builder.initializer = TopLocsInitializer()

sim = Simulation(
    geo=geo_library['maricopa_cbg_2019'](),
    ipm_builder=sdh_builder,
    mvm_builder=mm_library['centroids']()
)

out = sim.run(
    param={
        'alpha': [0.1, 0.3, 0.2],
        'theta': 0.1,
        'gamma': 0.1000022,
        'phi': 40,
        'move_control': 0.9,
        'infection_duration': 4.0,
        'immunity_duration': 90.0,
        'hospitalization_duration': 14.0,
        # Here are my Initializer args, same name as in the constructor.
        'infection_seed_size': 10_000,
        'infection_locations': 10,
        'infection_attribute': 'population'
    },
    start_date=date(2019, 1, 1),
    duration_days=1
)

# This is not very interesting output, but you get the idea.
out.prevalence[0, 0:10]

Compiled predef in 211.713 ms
Selected 10 locations by geo[population]:
   labels: ['040132168472' '040136103001' '040136109002' '040134226461'
 '040130822031' '040130506101' '040138160001' '040138152001'
 '040130405211' '040130610161']
  indices: [1389 2017 2037 1969  383  104 2445 2427   46  154]
   values: [6018 6235 6606 6867 7419 7808 9708 7788 8898 7929]


array([[1791,    0,    0,    0],
       [2007,    0,    0,    0],
       [1264,    0,    0,    0],
       [1139,    0,    0,    0],
       [1487,    0,    0,    0],
       [2501,    0,    0,    0],
       [2051,    0,    0,    0],
       [1356,    0,    0,    0],
       [1838,    0,    0,    0],
       [ 647,    0,    0,    0]])